In [23]:
######querying with Andrew's data - include error handling later (if clf doesn't exist for the page being queried)
test_page = 'BuzzFeed Animals'
ind = pages.index(test_page)
clf = page_clf[ind]

temp = lf_meta[~(lf_meta.page == test_page)]
temp = temp[:1000]
# temp = temp.sort_values('stats_date', ascending = False)[:1000]
query_titles = temp['title'].unique()

def get_title_fit(X, clf):
     return clf.predict_proba(X)

demo = pd.DataFrame()
demo['title'] = query_titles
temp = pd.DataFrame(clf.predict_proba(query_titles), columns=clf.classes_)
demo['prob'] = temp[1]

print("Top 10 links that are most relevant for FB ", test_page)
print("--------------")
for nam in demo.sort_values('prob', ascending = False)[:10]['title']:
    print(nam)

Top 10 links that are most relevant for FB  BuzzFeed Animals
--------------
This Quiz Will Determine If You Like Cats Or Dogs More
17 Photos That’ll Make Every Pet Parent Say “Same"
This Boy And His Dog's Sweet Friendship Will Make You Feel All The Feels
18 Puppies Who Are Up To No Good
19 Things Only Cat Owners Know To Be True
21 Puppies Who Won't Be Small For Long
Which Puppy Should Give You A Pep Talk?
Let These Puppies Decide What You Should Get For Lunch
12 GIFs Of People Dancing Guaranteed To Make You Smile
12 Motivational Posters That Are Way Too Real If You Own A Cat


In [18]:
###TASK 1 - LINKS (using Andrew's data)

import pandas as pd
# vf_stats = pd.read_csv('link_stats_jun20.csv')
andrew_df = pd.read_csv('/Users/eshwarchandrasekharan/Downloads/post_metadata_7-31-17.csv', error_bad_lines = False)
lf_meta = andrew_df
lf_meta['title'] = lf_meta['web_app_post.name']
lf_meta['page'] = lf_meta['display_name']
####
all_pages = lf_meta['page'].unique()

###which page is best fit for a given just the title - 
from sklearn.model_selection import KFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest
import numpy as np

pages = []
num_links = []
average_acc = []
average_pre = []
average_rec = []
average_f1 = []

cv_folds = 10
results = pd.DataFrame()
page_clf = []

df = lf_meta[['title', 'page']].dropna()

for test_page in all_pages:
#     test_page = "BuzzFeed"
    X_1 = df[df.page == test_page]
    X_0 = df[~(df.page == test_page)].sample(n=len(X_1))
    if len(X_1) < 2.5*cv_folds:
        continue
    X_1 = X_1[['title', 'page']]
    X_0 = X_0[['title', 'page']]
    X_all = pd.concat([X_1, X_0])
    
#     print(len(X_1), ":", len(X_0))
    def get_class(X, page):
        if X == page:
            return 1
        else:
            return 0
    X_all['class'] = X_all['page'].apply(get_class, page = test_page)
#     X_all = X_all.dropna()
    text_clf = Pipeline([
                         ('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
#                          ('featselect', SelectKBest(k = 100)),
                         ('clf', MultinomialNB()),
#                          ('clf', LinearSVC()),
#                          ('clf', RandomForestClassifier()),
#                          ('clf', LogisticRegression()),
    ])
    fold = 0
    kf = KFold(n_splits = cv_folds, shuffle = True)

    accuracy = []
    precision = []
    recall = []
    fscore = []

    X = X_all['title']
    y = X_all['class']

    for train_index, test_index in kf.split(X):
#         print("Fold = ", fold)
        fold+= 1
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        text_clf.fit(X_train, y_train)
        y_pred = text_clf.predict(X_test)

        acc = accuracy_score(y_test, y_pred)
        prec, rec, fs, sup = precision_recall_fscore_support(y_test, y_pred, average='macro')
        accuracy.append(acc)
        precision.append(prec)
        recall.append(rec)
        fscore.append(fs)
#     print("Page: ", test_page, "#links: ", len(X_1), "Accuracy: ", np.mean(accuracy), "Precision: ", np.mean(precision), "Recall: ", np.mean(recall), "Fscore:", np.mean(fscore))
    num_links.append(len(X_1))
    average_acc.append(np.mean(accuracy))
    average_pre.append(np.mean(precision))
    average_rec.append(np.mean(recall))
    average_f1.append(np.mean(fscore))
    pages.append(test_page)
    page_clf.append(text_clf)
results['page'] = pages
results['num_links'] = num_links
results['accuracy'] = average_acc
results['precision'] = average_pre
results['recall'] = average_rec
results['fscore'] = average_f1

results.mean()

### First page promotion - previous dataset
###Total data points: 85591
# num_pages: 85
# num_links    1448.000000
# accuracy        0.789186
# precision       0.811785
# recall          0.792267
# fscore          0.778326

#####test with Andrew's data (98637 rows)
# num_links    3287.900000
# accuracy        0.768146
# precision       0.774221
# recall          0.768273
# fscore          0.766238

b'Skipping line 18874: expected 9 fields, saw 10\nSkipping line 21002: expected 9 fields, saw 10\nSkipping line 21355: expected 9 fields, saw 11\n'


num_links    3287.900000
accuracy        0.770588
precision       0.776932
recall          0.770811
fscore          0.768698
dtype: float64

In [19]:
andrew_df.shape, lf_meta.shape

((98637, 11), (98637, 11))

In [1]:
###TASK 1 - LINKS
import pandas as pd
# vf_stats = pd.read_csv('link_stats_jun20.csv')
lf_meta = pd.read_csv('../models-links/radshift_links_2016_2017.csv')

all_pages = lf_meta['page'].unique()

###which page is best fit for a given just the title - 
from sklearn.model_selection import KFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest
import numpy as np

pages = []
num_links = []
average_acc = []
average_pre = []
average_rec = []
average_f1 = []

cv_folds = 10
results = pd.DataFrame()
page_clf = []

df = lf_meta[['title', 'page']].dropna()

for test_page in all_pages:
#     test_page = "BuzzFeed"
    X_1 = df[df.page == test_page]
    X_0 = df[~(df.page == test_page)].sample(n=len(X_1))
    if len(X_1) < 2.5*cv_folds:
        continue
    X_1 = X_1[['title', 'page']]
    X_0 = X_0[['title', 'page']]
    X_all = pd.concat([X_1, X_0])
    
#     print(len(X_1), ":", len(X_0))
    def get_class(X, page):
        if X == page:
            return 1
        else:
            return 0
    X_all['class'] = X_all['page'].apply(get_class, page = test_page)
#     X_all = X_all.dropna()
    text_clf = Pipeline([
                         ('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
#                          ('featselect', SelectKBest(k = 100)),
                         ('clf', MultinomialNB()),
#                          ('clf', LinearSVC()),
#                          ('clf', RandomForestClassifier()),
#                          ('clf', LogisticRegression()),
    ])
    fold = 0
    kf = KFold(n_splits = cv_folds, shuffle = True)

    accuracy = []
    precision = []
    recall = []
    fscore = []

    X = X_all['title']
    y = X_all['class']

    for train_index, test_index in kf.split(X):
#         print("Fold = ", fold)
        fold+= 1
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        text_clf.fit(X_train, y_train)
        y_pred = text_clf.predict(X_test)

        acc = accuracy_score(y_test, y_pred)
        prec, rec, fs, sup = precision_recall_fscore_support(y_test, y_pred, average='macro')
        accuracy.append(acc)
        precision.append(prec)
        recall.append(rec)
        fscore.append(fs)
#     print("Page: ", test_page, "#links: ", len(X_1), "Accuracy: ", np.mean(accuracy), "Precision: ", np.mean(precision), "Recall: ", np.mean(recall), "Fscore:", np.mean(fscore))
    num_links.append(len(X_1))
    average_acc.append(np.mean(accuracy))
    average_pre.append(np.mean(precision))
    average_rec.append(np.mean(recall))
    average_f1.append(np.mean(fscore))
    pages.append(test_page)
    page_clf.append(text_clf)
results['page'] = pages
results['num_links'] = num_links
results['accuracy'] = average_acc
results['precision'] = average_pre
results['recall'] = average_rec
results['fscore'] = average_f1

results.mean()

### First page promotion
###Total data points: 85591
# num_pages: 85
# num_links    1448.000000
# accuracy        0.789186
# precision       0.811785
# recall          0.792267
# fscore          0.778326

#####test with Andrew's data (98637 rows)
# num_links    3287.900000
# accuracy        0.768146
# precision       0.774221
# recall          0.768273
# fscore          0.766238

/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


num_links    1448.000000
accuracy        0.789857
precision       0.815148
recall          0.792735
fscore          0.779524
dtype: float64

In [10]:

#####Query by page and title of link - Should a link with this title be posted on this BF page?
# q_page = "LOLA"
q_page = "BuzzFeed Quiz"
# q_page = "BuzzFeed Japan"

q_test = ["5 DIY Soap Ideas"]
# q_test = ['Camping Meal Prep Hacks']
# q_test = ["People Try Kardashian Beauty Secrets"]
# q_test = ['This Restaurant In Mumbai Is Serving Up Goth Tikka']  ###list of titles you wanna query
# q_test = ["Find Out What your spirit animal is?"]  ###list of titles you wanna query

max_prob = 0
max_page = ""

print("Finding best first page for item titled: ", q_test)

for pg in all_pages:
    try:
        clf = page_clf[pages.index(pg)]
    except:
#         print("Woops, the model for page ", pg, " is still in the works!")
        continue
        
    prob = clf.predict_proba(q_test)[0][1]
    print(pg, prob)
    if prob > max_prob:
        max_prob = prob
        max_page = pg
    
print("Best fit for this link would be: ", max_page, "; prob = ", max_prob)

Finding best first page for item titled:  ['5 DIY Soap Ideas']
BuzzFeed Partner 0.829435684612
Cheeky 0.225256428384
BuzzFeed DIY 0.888881222809
BuzzFeed Entertainment 0.249794354286
BuzzFeed World 0.24772764345
BuzzFeed Rewind 0.236342542472
BuzzFeed Books 0.417053300364
Cocoa Butter 0.187122964563
BuzzFeed Health 0.477480940329
BuzzFeed 0.392046522428
BuzzFeed Celeb 0.173027283558
BuzzFeed Science 0.273454556059
BuzzFeed News 0.0868352148482
BuzzFeed Parents 0.631152267097
BuzzFeed Australia 0.293100657078
BuzzFeed Style 0.664991707289
BuzzFeed Community 0.646891689936
BuzzFeed Quiz 0.172516538375
BuzzFeed Politics 0.142905670186
BuzzFeed Animals 0.400222672277
Buy Me That 0.396477915068
BuzzFeed Food 0.714112699956
BuzzFeed UK 0.491900620884
SOML 0.585315545586
BuzzFeed LGBT 0.387019351993
BuzzFeed Tech 0.144868570324
Obsessed by BuzzFeed 0.178198090614
Tasty 0.591989285159
BuzzFeed Video 0.216898516493
BuzzFeed Canada 0.6614710383
Best fit for this link would be:  BuzzFeed DIY ; pr

In [81]:
test_page = 'Tasty'
ind = pages.index(test_page)
clf = page_clf[ind]

temp = lf_meta[~(lf_meta.page == test_page)]
temp = temp.sort_values('stats_date', ascending = False)[:1000]
query_titles = temp['title'].unique()

def get_title_fit(X, clf):
     return clf.predict_proba(X)

demo = pd.DataFrame()
demo['title'] = query_titles
temp = pd.DataFrame(clf.predict_proba(query_titles), columns=clf.classes_)
demo['prob'] = temp[1]

print("Top 10 links that are most relevant for FB ", test_page)
print("--------------")
for nam in demo.sort_values('prob', ascending = False)[:10]['title']:
    print(nam)

Top 10 links that are most relevant for FB  Tasty
--------------
You Can Only Eat Pasta If You Get 12/15 In This Quiz
7 Dinners Under $10 You Should Make This Week
Only A True Pizza Expert Can Identify Which Restaurant These Pizzas Are From Based On A Picture
7 Small Ways To Actually Get Your Home In Order
How Much Do You Actually Know About Cooking Eggs?
23 Cheap Products That'll Actually Keep Your Kitchen Organized
We Know Which Job You Should Actually Have Based On Your Apartment Preferences
Pick Seven Of Your Favorite Foods And We'll Reveal What Your Life Will Be Like In 10 Years
Your Proposal Preferences Will Reveal When You'll Get Engaged
Tell Us Eight Of Your Food Preferences And We'll Reveal Where You Should Move To


In [17]:
test_page = 'BuzzFeed Animals'
ind = pages.index(test_page)
clf = page_clf[ind]

temp = lf_meta[~(lf_meta.page == test_page)]
temp = temp[:1000]
temp = temp.sort_values('stats_date', ascending = False)[:1000]
query_titles = temp['title'].unique()

def get_title_fit(X, clf):
     return clf.predict_proba(X)

demo = pd.DataFrame()
demo['title'] = query_titles
temp = pd.DataFrame(clf.predict_proba(query_titles), columns=clf.classes_)
demo['prob'] = temp[1]

print("Top 10 links that are most relevant for FB ", test_page)
print("--------------")
for nam in demo.sort_values('prob', ascending = False)[:10]['title']:
    print(nam)

Top 10 links that are most relevant for FB  BuzzFeed Animals
--------------
This Quiz Will Determine If You Like Cats Or Dogs More
18 Puppies Who Are Up To No Good
17 Photos That’ll Make Every Pet Parent Say “Same"
19 Things Only Cat Owners Know To Be True
The Starving Sea Lion Pup Found Sleeping In A Restaurant Has A Good Chance Of Surviving
This Boy And His Dog's Sweet Friendship Will Make You Feel All The Feels
18 Small Things That Will Make You Smile
21 Puppies Who Won't Be Small For Long
This Adorable House-Broken Bison Found Her Forever Home
17 Photos That Will Make Any Broke Person Laugh Then Cry


In [72]:
results

,page,num_links,accuracy,precision,recall,fscore
0,BuzzFeed Rewind,2106,0.769468,0.804743,0.770652,0.762497
1,BuzzFeed UK,4096,0.670780,0.704175,0.671220,0.656821
2,BuzzFeed,10435,0.617729,0.663590,0.617901,0.588998
3,BuzzFeed Food,2855,0.815061,0.837122,0.815317,0.811916
4,BuzzFeed Entertainment,2791,0.792731,0.807137,0.792996,0.790220
5,BuzzFeed Books,1978,0.747216,0.783619,0.747919,0.738632
6,BuzzFeed Science,269,0.778826,0.781032,0.781168,0.776306
7,BuzzFeed Parents,940,0.703191,0.764443,0.703677,0.684662
8,Tasty,972,0.774716,0.813309,0.774685,0.766898
9,BuzzFeed Australia,5360,0.641978,0.679455,0.642291,0.622433


In [73]:
test_page = 'BuzzFeed Style'
ind = pages.index(test_page)
clf = page_clf[ind]

temp = lf_meta[~(lf_meta.page == test_page)]
temp = temp.sort_values('stats_date', ascending = False)[:1000]
# temp = lf_meta[['title', 'page']]
query_titles = temp['title'].unique()

def get_title_fit(X, clf):
     return clf.predict_proba(X)

demo = pd.DataFrame()
demo['title'] = query_titles
temp = pd.DataFrame(clf.predict_proba(query_titles), columns=clf.classes_)
demo['prob'] = temp[1]
print("Top 10 links that are most relevant for FB ", test_page)
print("--------------")
for nam in demo.sort_values('prob', ascending = False)[:20]['title']:
    print(nam)

Top 10 links that are most relevant for FB  BuzzFeed Style
--------------
All The Best Deals On Amazon Today
All The Best Deals On The Internet This Weekend
29 Of The Best Beauty Products To Buy At Target
29 Gorgeous And Cheap Dresses To Wear To An Outdoor Wedding
How Would You Wear These Red High Heels?
11 Beauty Products That Will Actually Take Off All Your Makeup
25 Gorgeous Nail Polishes People Actually Swear By
26 Awesome Beauty Products You Didn't Know You Could Get At Urban Outfitters
26 Of The Best Places To Buy Last Minute Gifts
Here's What Amazon Prom Dresses Look Like On Non-Models
Here's What Forever 21's New Bridesmaid Dresses Look Like IRL
Create A Sitcom And We'll Reveal Your Hair And Eye Color
8 Life-Changing Tech Products You Need ASAP
19 Photos That Prove Natural Hair Doesn't Need Twist-Outs To Look Beautiful
21 Beautiful Ring Sets You'll Want On Your Fingers ASAP
23 Easy Hacks That'll Make Every Lazy Girl's Life So Much Easier
Kim Kardashian Is Launching Her Own Make

In [75]:
test_page = 'BuzzFeed News'
ind = pages.index(test_page)
clf = page_clf[ind]

temp = lf_meta[~(lf_meta.page == test_page)]
# temp = lf_meta[~(lf_meta.page == test_page)]
temp = temp.sort_values('stats_date', ascending = False)[:1000]
# temp = lf_meta[['title', 'page']]
query_titles = temp['title'].unique()

def get_title_fit(X, clf):
     return clf.predict_proba(X)

demo = pd.DataFrame()
demo['title'] = query_titles
temp = pd.DataFrame(clf.predict_proba(query_titles), columns=clf.classes_)
demo['prob'] = temp[1]

print("Top 10 links that are most relevant for FB ", test_page)
print("--------------")
for nam in demo.sort_values('prob', ascending = False)[:20]['title']:
    print(nam)

Top 10 links that are most relevant for FB  BuzzFeed News
--------------
Police Officer Found Not Guilty In The Fatal Shooting Of Philando Castile
Live Updates: Finsbury Park Terror Suspect Remains In Custody As Police Obtain Warrant For Further Detention
Six People Have Resigned From Trump's HIV/AIDS Advisory Council Because He &quot;Doesn't Care&quot;
Kochi Metro Normalises Trans People At Work With This Message To Passengers
An Aussie Bloke Allegedly Tunnelled His Way Out Of A Bali Jail Just Months Before His Release Date
Doctors Dispute North Korea's Reason For US Student Being Comatose
Vice President Pence Has Hired His Own Lawyer In Connection With The Special Counsel Probe
New Zealand Has The Highest Adolescent Suicide Rate Of All Wealthy Countries, Says New Report
Trump Rolls Back Obama’s Cuba Policies On Travel And Trade
Steve Harvey Gets Roasted On Twitter After Tasteless Flint Water Crisis Joke
This Couple Turned 42 On The Same Day And Did The Cutest Birthday Photo Shoot
Whe

In [66]:
temp = lf_meta[~(lf_meta.page == test_page)]
temp[temp.page == 'BuzzFeed News']

,external_id,created_at,page,post_type,buzz_id,uri,title,stats_ts,stats_date,consumptions_by_type__link_clicks,...,stories_by_action_type__comment,stories_by_action_type__like,stories_by_action_type__share,engaged_fan,fan_reach,impressions,impressions_fan,twoday_stats_date,twoday_consumptions_by_type__link_clicks,twoday_stories_by_action_type__share


In [ ]:
Build A Wardrobe From Forever 21 And We will Guess Your Signature Style;Which Game Of Thrones Character Are You Based On Your Taste In Food?;18 GIFs That Sum Up Your Relationship With Summer 

In [29]:
###manny code testing

import logging
import sys
import numpy as np
import pandas as pd

# sklearn
from sklearn import linear_model
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import scipy.sparse

import nltk
from nltk.corpus import words
from nltk.tokenize import word_tokenize

# add static path to nltk data
nltk.data.path.append('/app/nltk_data')

"""
The links in this table are sorted based on the title fit (indicated by the probability or prob)
to the target page under consideration, and also the predicted first hour shares (if they are
posted on the target page). Out of the top (K * K) links, with respect to probability of fit to
the target page (sort by prob), only the top K links are returned as the output based on
predicted_first_hour_shares.
"""
def recommend_route(datasets=[], params={}):
    prev_next_stats = datasets[0]   # cross-posting info: current_page and next_page details for each [external_id]
    title_df = datasets[1]          # titles of BF links to train the title fit Bag-of-Words model
    explicit_double_check = datasets[2] # external_ids and display_names for all versions of a buzz/link

    test_page = params['test_page']
    lookback_limit = params['lookback_limit']
    max_suggestions = params['max_suggestions']

    print('Page: ', test_page)
    print('Until: ', lookback_limit)
    print('Max Suggestions: ', max_suggestions)
    print('')

    train_df = prev_next_stats
    train_df['page'] = train_df['display_name']
    train_df['title'] = train_df['name']
    train_df = train_df.fillna(0)
    all_pages = train_df['page'].unique()
    train_df['URL'] = "www.buzzfeed.com/" + train_df['user_name'] + "/" + train_df['uri']

    """
    type cast the previous and next page columns to categorical (makes life easy when
    doing pd.dummies later on)
    """
    train_df['next_page'] = train_df['next_page'].astype('category', categories = all_pages)
    train_df['page'] = train_df['page'].astype('category', categories = all_pages)

    """
    input variables: FB stats for a post on current (or previous)
    page in the 1st hour from posting
    """
    one_hour_features = [
        'consumptions_by_type__link_clicks', 'consumptions_by_type__other_clicks',
        'reactions_like_total', 'stories_by_action_type__comment', 'stories_by_action_type__like',
        'stories_by_action_type__share', 'engaged_fan', 'fan_reach', 'impressions',
        'impressions_fan',
    ]

    # linear regression model
    clf = linear_model.LinearRegression()
    # set features to train on
    page_infos = ['page', 'next_page']
    train_features = one_hour_features + page_infos
    for feat in one_hour_features:
        train_df[feat] = np.log(train_df[feat] + 1)

    train_df['clicks_bucket'] = np.log(train_df['next_clicks'] + 1)
    train_df['share_bucket'] = np.log(train_df['next_shares']+1)
    print('# of Data Points => ', len(train_df))

    skip_title_fit = 1
#     skip_title_fit = 0
    if skip_title_fit == 0:
        X = train_df[train_features  + ['title']]
    else: 
        X = train_df[train_features]

    y = train_df['share_bucket']

    # convert page_infos into dummies
    X = pd.get_dummies(data=X, columns=['page', 'next_page'])
    print('Converted page infos into dummies...')

    if skip_title_fit == 0:
        # pipeline to fit on title features!
        title_vect = Pipeline([
            ('vect', CountVectorizer()),
            ('tfidf', TfidfTransformer()),
        ])
        print('Ran pipeline on title features...')

        # get the title features and "hstack" them along with "one_hour_features": input variables
        X_tfidf = title_vect.fit_transform(X['title'])
        X = X.drop(['title'], axis = 1)
        X = scipy.sparse.hstack([X_tfidf, X])
        print('Parsed hstack...')

    # fit the regression model
    clf = clf.fit(X, y)
    print('Fit the regression model...')

    """
    check 2 conditions:
    1) creation date of link should be > lookback_limit (input parameter)
    2) current page of the link shouldn't be the test_page itself
    """
    test_df = train_df
    test_df = test_df[test_df['created_at'] > lookback_limit]
    # remove all buzz_ids which have atleast one external_id in the test_page
    def check_if_on_testpage(X):
      if test_page in explicit_double_check[explicit_double_check.buzz_id == X]['display_name'].unique():
        return 1
      else:
        return 0

    test_df['already_posted'] = test_df['buzz_id'].apply(check_if_on_testpage)
    # keep only buzzes that've not yet been posted on test_page
    test_df = test_df[test_df.already_posted == 0]
    test_df = test_df[~(test_df['page'] == test_page)]
    test_df = test_df[~(test_df['next_page'] == test_page)]
    print('Filtered already posted buzzes...')

    # manually assign next_page = test_page, since we are gonna use this dataframe to make predictions; convert the column to type "categorical"
    test_df['next_page'] = test_page
    test_df['next_page'] = test_df['next_page'].astype('category', categories = all_pages)

    X_test = test_df[train_features + ['title']]
    print('Assigned next page...')

    # convert current and next page names into Pandas dummy variables/columns (pd.get_dummies)
    X_test = pd.get_dummies(data = X_test, columns=['page', 'next_page'])
    print('Converted pages into pandas dummies...')
    
    if skip_title_fit == 0:
        # extract title features for the test dataframe
        X_tfidf = title_vect.transform(X_test['title'])
        X_test = X_test.drop(['title'], axis = 1)
        X_test = scipy.sparse.hstack([X_tfidf, X_test])
        print('Extracted title features for dataframe...')
    else:
        print('Skipping title fit...')
        
    # make predictions on a link's performance, if it were to be posted on the target page, given the perfoprmance in the previous page!
    test_df['predicted_shares'] = clf.predict(X_test)

    # convert the log-scaled predictions back to exponents for computing accuracy and errors!
    test_df['predicted_shares'] = (np.exp(test_df['predicted_shares'])-1).astype(int)
    test_df['predicted_first_hour_shares'] = test_df['predicted_shares']

    # list of columns to include in the output dataframe
    result_cols = ['page', 'title', 'buzz_id', 'predicted_first_hour_shares', 'URL', 'next_page', 'created_at']
    test_df = test_df[result_cols].drop_duplicates(subset=['buzz_id'], keep = 'last')

    df = title_df[['name', 'display_name']].dropna()
    df.columns = ['title', 'page']

    # training samples: class 1 = links from test_page; class 0 = random links that are from else where
    X_1 = df[df.page == test_page]
    X_0 = df[~(df.page == test_page)].sample(n=len(X_1))
    X_all = pd.concat([X_1, X_0])

    def get_class(X, page):
        return 1 if X == page else 0

    X_all['class'] = X_all['page'].apply(get_class, page=test_page)

    text_clf = Pipeline([
        ('vect', CountVectorizer(tokenizer=word_tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultinomialNB()),
    ])

    X = X_all['title']
    y = X_all['class']

    title_clf = text_clf.fit(X, y)
    temp = test_df

    pred = pd.DataFrame(title_clf.predict_proba(temp['title']), columns=title_clf.classes_)
    temp['prob'] = pred[1].values

    print('')
    print('Top ', max_suggestions, ' links that are most relevant for FB ', test_page, '; (test sample size: ', len(temp), ')')
    print('--------------')
    temp = temp.sort_values('prob', ascending = False)[:max_suggestions*10]
    temp['identifier'] = '{page}_{buzz_id}_{next_page}_{shares}'.format(
        page=temp['page'].astype(str),
        buzz_id=temp['buzz_id'].map(str),
        next_page=temp['next_page'].astype(str),
        shares=temp['predicted_first_hour_shares'].map(str)
    )

#     temp = temp.sort_values('predicted_first_hour_shares', ascending = False)[:max_suggestions]
    # The links in this table are sorted only based on the title fit (indicated by the probability or prob) to the target page under consideration.
    temp = temp.sort_values('prob', ascending = False)[:max_suggestions]
    print(temp)


if __name__ == "__main__":
    filepat = "../../mono/proto_recommendation_api/"
    query_1 = pd.read_csv(filepat + 'fixtures/routing_query_1.csv', encoding='utf-8', engine='python')
    print('Loaded query 1...')
    title_sources = pd.read_csv(filepat + 'fixtures/routing_title_sources.csv', encoding='utf-8', engine='python')
    print('Loaded title sources...')
    explicit_dc = pd.read_csv(filepat + 'fixtures/routing_explicit_double_check.csv', encoding='utf-8', engine='python')
    print('Loaded explicit double check...')
    print('')
    print('=> RUNNING RECOMMENDATIONS')
    print('')

    fake_data_sets = [query_1, title_sources, explicit_dc]
    fake_params = {
        'test_page': 'BuzzFeed Animals',
        'lookback_limit': '2017-01-01',
        'max_suggestions': 20,
    }
    recommend_route(datasets=fake_data_sets, params=fake_params)

Loaded query 1...
Loaded title sources...
Loaded explicit double check...

=> RUNNING RECOMMENDATIONS

Page:  BuzzFeed Animals
Until:  2017-01-01
Max Suggestions:  20

# of Data Points =>  46957
Converted page infos into dummies...
Ran pipeline on title features...
Parsed hstack...
Fit the regression model...
Filtered already posted buzzes...
Assigned next page...
Converted pages into pandas dummies...
Extracted title features for dataframe...

Top  20  links that are most relevant for FB  BuzzFeed Animals ; (test sample size:  16907 )
--------------
                     page                                              title  \
39497     BuzzFeed Canada  11 Adorable Animals That Are Way Better At Sav...   
44765            BuzzFeed  19 Wholesome Boyfriends Who Are So Damn Pure I...   
22877            BuzzFeed   16 Times Tumblr Was Too Pure For This Good Earth   
42868            BuzzFeed  21 Tweets That Are Really Weird But Will Make ...   
35357         BuzzFeed UK  11 Happy Little 

In [30]:
print(1)

1


In [33]:
###manny code testing

import logging
import sys
import numpy as np
import pandas as pd

# sklearn
from sklearn import linear_model
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import scipy.sparse

import nltk
from nltk.corpus import words
from nltk.tokenize import word_tokenize

# add static path to nltk data
nltk.data.path.append('/app/nltk_data')

"""
The links in this table are sorted based on the title fit (indicated by the probability or prob)
to the target page under consideration, and also the predicted first hour shares (if they are
posted on the target page). Out of the top (K * K) links, with respect to probability of fit to
the target page (sort by prob), only the top K links are returned as the output based on
predicted_first_hour_shares.
"""
def recommend_route(datasets=[], params={}):
    prev_next_stats = datasets[0]   # cross-posting info: current_page and next_page details for each [external_id]
    title_df = datasets[1]          # titles of BF links to train the title fit Bag-of-Words model
    explicit_double_check = datasets[2] # external_ids and display_names for all versions of a buzz/link

    test_page = params['test_page']
    lookback_limit = params['lookback_limit']
    max_suggestions = params['max_suggestions']

    print('Page: ', test_page)
    print('Until: ', lookback_limit)
    print('Max Suggestions: ', max_suggestions)
    print('')

    train_df = prev_next_stats
    train_df['page'] = train_df['display_name']
    train_df['title'] = train_df['name']
    train_df = train_df.fillna(0)
    all_pages = train_df['page'].unique()
    train_df['URL'] = "www.buzzfeed.com/" + train_df['user_name'] + "/" + train_df['uri']

    """
    type cast the previous and next page columns to categorical (makes life easy when
    doing pd.dummies later on)
    """
    train_df['next_page'] = train_df['next_page'].astype('category', categories = all_pages)
    train_df['page'] = train_df['page'].astype('category', categories = all_pages)

    """
    input variables: FB stats for a post on current (or previous)
    page in the 1st hour from posting
    """
    one_hour_features = [
        'consumptions_by_type__link_clicks', 'consumptions_by_type__other_clicks',
        'reactions_like_total', 'stories_by_action_type__comment', 'stories_by_action_type__like',
        'stories_by_action_type__share', 'engaged_fan', 'fan_reach', 'impressions',
        'impressions_fan',
    ]

    # linear regression model
    clf = linear_model.LinearRegression()
    # set features to train on
    page_infos = ['page', 'next_page']
    train_features = one_hour_features + page_infos
    for feat in one_hour_features:
        train_df[feat] = np.log(train_df[feat] + 1)

    train_df['clicks_bucket'] = np.log(train_df['next_clicks'] + 1)
    train_df['share_bucket'] = np.log(train_df['next_shares']+1)
    print('# of Data Points => ', len(train_df))

#     skip_title_fit = 1
    skip_title_fit = 0
    if skip_title_fit == 0:
        X = train_df[train_features  + ['title']]
    else: 
        X = train_df[train_features]

    y = train_df['share_bucket']

    # convert page_infos into dummies
    X = pd.get_dummies(data=X, columns=['page', 'next_page'])
    print('Converted page infos into dummies...')

    if skip_title_fit == 0:
        # pipeline to fit on title features!
        title_vect = Pipeline([
            ('vect', CountVectorizer()),
            ('tfidf', TfidfTransformer()),
        ])
        print('Ran pipeline on title features...')

        # get the title features and "hstack" them along with "one_hour_features": input variables
        X_tfidf = title_vect.fit_transform(X['title'])
        X = X.drop(['title'], axis = 1)
        X = scipy.sparse.hstack([X_tfidf, X])
        print('Parsed hstack...')

    # fit the regression model
    clf = clf.fit(X, y)
    print('Fit the regression model...')

    """
    check 2 conditions:
    1) creation date of link should be > lookback_limit (input parameter)
    2) current page of the link shouldn't be the test_page itself
    """
    test_df = train_df
    test_df = test_df[test_df['created_at'] > lookback_limit]
    # remove all buzz_ids which have atleast one external_id in the test_page
    def check_if_on_testpage(X):
      if test_page in explicit_double_check[explicit_double_check.buzz_id == X]['display_name'].unique():
        return 1
      else:
        return 0

    test_df['already_posted'] = test_df['buzz_id'].apply(check_if_on_testpage)
    # keep only buzzes that've not yet been posted on test_page
#     test_df = test_df[test_df.already_posted == 0]
    test_df = test_df[~(test_df['page'] == test_page)]
    test_df = test_df[~(test_df['next_page'] == test_page)]
    print('Filtered already posted buzzes...')

    # manually assign next_page = test_page, since we are gonna use this dataframe to make predictions; convert the column to type "categorical"
    test_df['next_page'] = test_page
    test_df['next_page'] = test_df['next_page'].astype('category', categories = all_pages)

    if skip_title_fit == 0:
        X_test = test_df[train_features + ['title']]
    else:
        X_test = test_df[train_features]

    print('Assigned next page...')

    # convert current and next page names into Pandas dummy variables/columns (pd.get_dummies)
    X_test = pd.get_dummies(data = X_test, columns=['page', 'next_page'])
    print('Converted pages into pandas dummies...')
    
    if skip_title_fit == 0:
        # extract title features for the test dataframe
        X_tfidf = title_vect.transform(X_test['title'])
        X_test = X_test.drop(['title'], axis = 1)
        X_test = scipy.sparse.hstack([X_tfidf, X_test])
        print('Extracted title features for dataframe...')
    else:
        print('Skipping title fit...')
        
    # make predictions on a link's performance, if it were to be posted on the target page, given the perfoprmance in the previous page!
    test_df['predicted_shares'] = clf.predict(X_test)

    # convert the log-scaled predictions back to exponents for computing accuracy and errors!
    test_df['predicted_shares'] = (np.exp(test_df['predicted_shares'])-1).astype(int)
    test_df['predicted_first_hour_shares'] = test_df['predicted_shares']

    # list of columns to include in the output dataframe
    result_cols = ['page', 'title', 'buzz_id', 'predicted_first_hour_shares', 'URL', 'next_page', 'created_at']
    test_df = test_df[result_cols].drop_duplicates(subset=['buzz_id'], keep = 'last')

    df = title_df[['name', 'display_name']].dropna()
    df.columns = ['title', 'page']

    # training samples: class 1 = links from test_page; class 0 = random links that are from else where
    X_1 = df[df.page == test_page]
    X_0 = df[~(df.page == test_page)].sample(n=len(X_1))
    X_all = pd.concat([X_1, X_0])

    def get_class(X, page):
        return 1 if X == page else 0

    X_all['class'] = X_all['page'].apply(get_class, page=test_page)

    text_clf = Pipeline([
        ('vect', CountVectorizer(tokenizer=word_tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultinomialNB()),
    ])

    X = X_all['title']
    y = X_all['class']

    title_clf = text_clf.fit(X, y)
    temp = test_df

    pred = pd.DataFrame(title_clf.predict_proba(temp['title']), columns=title_clf.classes_)
    temp['prob'] = pred[1].values

    print('')
    print('Top ', max_suggestions, ' links that are most relevant for FB ', test_page, '; (test sample size: ', len(temp), ')')
    print('--------------')
    temp = temp.sort_values('prob', ascending = False)[:max_suggestions*10]
    temp['identifier'] = '{page}_{buzz_id}_{next_page}_{shares}'.format(
        page=temp['page'].astype(str),
        buzz_id=temp['buzz_id'].map(str),
        next_page=temp['next_page'].astype(str),
        shares=temp['predicted_first_hour_shares'].map(str)
    )

#     temp = temp.sort_values('predicted_first_hour_shares', ascending = False)[:max_suggestions]
    # The links in this table are sorted only based on the title fit (indicated by the probability or prob) to the target page under consideration.
    temp = temp.sort_values('prob', ascending = False)[:max_suggestions]
    print(temp)


if __name__ == "__main__":
    filepat = "../../mono/proto_recommendation_api/"
    query_1 = pd.read_csv(filepat + 'fixtures/routing_query_1.csv', encoding='utf-8', engine='python')
    print('Loaded query 1...')
    title_sources = pd.read_csv(filepat + 'fixtures/routing_title_sources.csv', encoding='utf-8', engine='python')
    print('Loaded title sources...')
    explicit_dc = pd.read_csv(filepat + 'fixtures/routing_explicit_double_check.csv', encoding='utf-8', engine='python')
    print('Loaded explicit double check...')
    print('')
    print('=> RUNNING RECOMMENDATIONS')
    print('')

    fake_data_sets = [query_1, title_sources, explicit_dc]
    fake_params = {
        'test_page': 'BuzzFeed Food',
        'lookback_limit': '2017-01-01',
        'max_suggestions': 20,
    }
    recommend_route(datasets=fake_data_sets, params=fake_params)

Loaded query 1...
Loaded title sources...
Loaded explicit double check...

=> RUNNING RECOMMENDATIONS

Page:  BuzzFeed Food
Until:  2017-01-01
Max Suggestions:  20

# of Data Points =>  46957
Converted page infos into dummies...
Fit the regression model...
Filtered already posted buzzes...
Assigned next page...
Converted pages into pandas dummies...
Skipping title fit...

Top  20  links that are most relevant for FB  BuzzFeed Food ; (test sample size:  15950 )
--------------
                       page  \
28271         BuzzFeed Quiz   
10332         BuzzFeed Quiz   
13412        BuzzFeed India   
5114          BuzzFeed Quiz   
33091         BuzzFeed Quiz   
18833      BuzzFeed Parents   
19706         BuzzFeed Quiz   
11161         BuzzFeed Quiz   
17873              BuzzFeed   
2201       BuzzFeed Parents   
45374  BuzzFeed Philippines   
35998              BuzzFeed   
37644       BuzzFeed Health   
43360         BuzzFeed Quiz   
45050           BuzzFeed UK   
33810       BuzzFeed Can

In [37]:
###manny code testing

import logging
import sys
import numpy as np
import pandas as pd

# sklearn
from sklearn import linear_model
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import scipy.sparse

import nltk
from nltk.corpus import words
from nltk.tokenize import word_tokenize

# add static path to nltk data
nltk.data.path.append('/app/nltk_data')

"""
The links in this table are sorted based on the title fit (indicated by the probability or prob)
to the target page under consideration, and also the predicted first hour shares (if they are
posted on the target page). Out of the top (K * K) links, with respect to probability of fit to
the target page (sort by prob), only the top K links are returned as the output based on
predicted_first_hour_shares.
"""
def recommend_route(datasets=[], params={}):
    prev_next_stats = datasets[0]   # cross-posting info: current_page and next_page details for each [external_id]
    title_df = datasets[1]          # titles of BF links to train the title fit Bag-of-Words model
    explicit_double_check = datasets[2] # external_ids and display_names for all versions of a buzz/link

    test_page = params['test_page']
    lookback_limit = params['lookback_limit']
    max_suggestions = params['max_suggestions']

    print('Page: ', test_page)
    print('Until: ', lookback_limit)
    print('Max Suggestions: ', max_suggestions)
    print('')

    train_df = prev_next_stats
    train_df['page'] = train_df['display_name']
    train_df['title'] = train_df['name']
    train_df = train_df.fillna(0)
    all_pages = train_df['page'].unique()
    train_df['URL'] = "www.buzzfeed.com/" + train_df['user_name'] + "/" + train_df['uri']

    """
    type cast the previous and next page columns to categorical (makes life easy when
    doing pd.dummies later on)
    """
    train_df['next_page'] = train_df['next_page'].astype('category', categories = all_pages)
    train_df['page'] = train_df['page'].astype('category', categories = all_pages)

    """
    input variables: FB stats for a post on current (or previous)
    page in the 1st hour from posting
    """
    one_hour_features = [
        'consumptions_by_type__link_clicks', 'consumptions_by_type__other_clicks',
        'reactions_like_total', 'stories_by_action_type__comment', 'stories_by_action_type__like',
        'stories_by_action_type__share', 'engaged_fan', 'fan_reach', 'impressions',
        'impressions_fan',
    ]

    # linear regression model
    clf = linear_model.LinearRegression()
    # set features to train on
    page_infos = ['page', 'next_page']
    train_features = one_hour_features + page_infos
    for feat in one_hour_features:
        train_df[feat] = np.log(train_df[feat] + 1)

    train_df['clicks_bucket'] = np.log(train_df['next_clicks'] + 1)
    train_df['share_bucket'] = np.log(train_df['next_shares']+1)
    print('# of Data Points => ', len(train_df))

#     skip_title_fit = 1
    skip_title_fit = 0
    if skip_title_fit == 0:
        X = train_df[train_features  + ['title']]
    else: 
        X = train_df[train_features]

    y = train_df['share_bucket']

    # convert page_infos into dummies
    X = pd.get_dummies(data=X, columns=['page', 'next_page'])
    print('Converted page infos into dummies...')

    if skip_title_fit == 0:
        # pipeline to fit on title features!
        title_vect = Pipeline([
            ('vect', CountVectorizer()),
            ('tfidf', TfidfTransformer()),
        ])
        print('Ran pipeline on title features...')

        # get the title features and "hstack" them along with "one_hour_features": input variables
        X_tfidf = title_vect.fit_transform(X['title'])
        X = X.drop(['title'], axis = 1)
        X = scipy.sparse.hstack([X_tfidf, X])
        print('Parsed hstack...')

    # fit the regression model
    clf = clf.fit(X, y)
    print('Fit the regression model...')

    """
    check 2 conditions:
    1) creation date of link should be > lookback_limit (input parameter)
    2) current page of the link shouldn't be the test_page itself
    """
    test_df = train_df
    test_df = test_df[test_df['created_at'] > lookback_limit]
    # remove all buzz_ids which have atleast one external_id in the test_page
    def check_if_on_testpage(X):
      if test_page in explicit_double_check[explicit_double_check.buzz_id == X]['display_name'].unique():
        return 1
      else:
        return 0

    test_df['already_posted'] = test_df['buzz_id'].apply(check_if_on_testpage)
    # keep only buzzes that've not yet been posted on test_page
    test_df = test_df[test_df.already_posted == 0]
    test_df = test_df[~(test_df['page'] == test_page)]
    test_df = test_df[~(test_df['next_page'] == test_page)]
    print('Filtered already posted buzzes...')

    # manually assign next_page = test_page, since we are gonna use this dataframe to make predictions; convert the column to type "categorical"
    test_df['next_page'] = test_page
    test_df['next_page'] = test_df['next_page'].astype('category', categories = all_pages)

    if skip_title_fit == 0:
        X_test = test_df[train_features + ['title']]
    else:
        X_test = test_df[train_features]

    print('Assigned next page...')

    # convert current and next page names into Pandas dummy variables/columns (pd.get_dummies)
    X_test = pd.get_dummies(data = X_test, columns=['page', 'next_page'])
    print('Converted pages into pandas dummies...')
    
    if skip_title_fit == 0:
        # extract title features for the test dataframe
        X_tfidf = title_vect.transform(X_test['title'])
        X_test = X_test.drop(['title'], axis = 1)
        X_test = scipy.sparse.hstack([X_tfidf, X_test])
        print('Extracted title features for dataframe...')
    else:
        print('Skipping title fit...')
        
    # make predictions on a link's performance, if it were to be posted on the target page, given the perfoprmance in the previous page!
    test_df['predicted_shares'] = clf.predict(X_test)

    # convert the log-scaled predictions back to exponents for computing accuracy and errors!
    test_df['predicted_shares'] = (np.exp(test_df['predicted_shares'])-1).astype(int)
    test_df['predicted_first_hour_shares'] = test_df['predicted_shares']

    # list of columns to include in the output dataframe
    result_cols = ['page', 'title', 'buzz_id', 'predicted_first_hour_shares', 'URL', 'next_page', 'created_at']
    test_df = test_df[result_cols].drop_duplicates(subset=['buzz_id'], keep = 'last')

    df = title_df[['name', 'display_name']].dropna()
    df.columns = ['title', 'page']

    # training samples: class 1 = links from test_page; class 0 = random links that are from else where
    X_1 = df[df.page == test_page]
    X_0 = df[~(df.page == test_page)].sample(n=len(X_1))
    X_all = pd.concat([X_1, X_0])

    def get_class(X, page):
        return 1 if X == page else 0

    X_all['class'] = X_all['page'].apply(get_class, page=test_page)

    text_clf = Pipeline([
        ('vect', CountVectorizer(tokenizer=word_tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultinomialNB()),
    ])

    X = X_all['title']
    y = X_all['class']

    title_clf = text_clf.fit(X, y)
    temp = test_df

    pred = pd.DataFrame(title_clf.predict_proba(temp['title']), columns=title_clf.classes_)
    temp['prob'] = pred[1].values

    print('')
    print('Top ', max_suggestions, ' links that are most relevant for FB ', test_page, '; (test sample size: ', len(temp), ')')
    print('--------------')
    temp = temp.sort_values('prob', ascending = False)[:max_suggestions*10]
    temp['identifier'] = '{page}_{buzz_id}_{next_page}_{shares}'.format(
        page=temp['page'].astype(str),
        buzz_id=temp['buzz_id'].map(str),
        next_page=temp['next_page'].astype(str),
        shares=temp['predicted_first_hour_shares'].map(str)
    )

#     temp = temp.sort_values('predicted_first_hour_shares', ascending = False)[:max_suggestions]
    # The links in this table are sorted only based on the title fit (indicated by the probability or prob) to the target page under consideration.
    temp = temp.sort_values('prob', ascending = False)[:max_suggestions]
    print(temp)


if __name__ == "__main__":
    filepat = "../../mono/proto_recommendation_api/"
    query_1 = pd.read_csv(filepat + 'fixtures/routing_query_1.csv', encoding='utf-8', engine='python')
    print('Loaded query 1...')
    title_sources = pd.read_csv(filepat + 'fixtures/routing_title_sources.csv', encoding='utf-8', engine='python')
    print('Loaded title sources...')
    explicit_dc = pd.read_csv(filepat + 'fixtures/routing_explicit_double_check.csv', encoding='utf-8', engine='python')
    print('Loaded explicit double check...')
    print('')
    print('=> RUNNING RECOMMENDATIONS')
    print('')

    fake_data_sets = [query_1, title_sources, explicit_dc]
    fake_params = {4
        'test_page': 'Nifty',
        'lookback_limit': '2017-01-01',
        'max_suggestions': 20,
    }
    recommend_route(datasets=fake_data_sets, params=fake_params)

Loaded query 1...
Loaded title sources...
Loaded explicit double check...

=> RUNNING RECOMMENDATIONS

Page:  Nifty
Until:  2017-01-01
Max Suggestions:  20

# of Data Points =>  46957
Converted page infos into dummies...
Ran pipeline on title features...
Parsed hstack...
Fit the regression model...
Filtered already posted buzzes...
Assigned next page...
Converted pages into pandas dummies...
Extracted title features for dataframe...

Top  20  links that are most relevant for FB  Nifty ; (test sample size:  17579 )
--------------
                     page                                              title  \
31273     BuzzFeed Health  23 Birth Control Alarms That Will Make You Lau...   
39773        BuzzFeed DIY  30 Cleaning Tips That'll Make Your Life So Muc...   
35913        BuzzFeed DIY   23 Things That'll Make Your House So Much Homier   
28884            BuzzFeed  23 Tweets About Amazon Prime Day That'll Make ...   
5976   BuzzFeed Australia  21 Things That Will Make Australians F

In [41]:
# external_id = 1318800798260799_2282584891882380: actual 1st hour shares = 325,   Identifier: BuzzFeed_4461993_BuzzFeed Video_389
# external_id = 1318800798260799_2282590138548522: actual 1st hour shares = 248,   Identifier: BuzzFeed Australia_4494602_BuzzFeed Video_408
y_test = [325, 248]
y_pred = [389, 408]
from sklearn import metrics
accuracy = metrics.r2_score(y_test, y_pred)
#     print("Cross-Predicted Accuracy (R2):", accuracy)
from sklearn.metrics import mean_absolute_error
# print("Mean Absolute Error: ", mean_absolute_error(y, predictions))
error_percent = mean_absolute_error(y_test, y_pred)/np.mean(y_test)

In [42]:
accuracy, error_percent

(-9.0172035756451336, 0.39092495636998253)

In [43]:
mean_absolute_error(y_test, y_pred)

112.0

In [45]:
np.mean(y_test)

286.5